# Data preparation

## Purpose
This notebook prepares the data to be consumed by the next step ([3_modeling_suto](3_modeling_suto.ipynb)).

## Methodology

## WIP - improvements

## Results

## Suggested next steps


# Setup

## Library import
We import all the required Python libraries

In [36]:
import os
from typing import List

# Data manipulation
import pandas as pd
import numpy as np

# Visualizations
import matplotlib as plt
from pandas_profiling import ProfileReport
import plotly
import plotly.graph_objs as go
import plotly.offline as ply
import seaborn as sns

os.chdir('../')
from src.utils.data_describe import breve_descricao, serie_nulos, cardinalidade
os.chdir('./notebooks/')

# Options for pandas
# pd.options.display.max_columns = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# pd.options.display.max_rows = 120

plotly.offline.init_notebook_mode(connected=True)

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

# Parameter definition
We set all relevant parameters for our notebook. By convention, parameters are uppercase, while all the 
other variables follow Python's guidelines.

In [84]:
RAW_FOLDER = '../data/raw/'
INTERIM_FOLDER = '../data/interim/'
REPORTS_FOLDER = '../reports/'
RANDOM_STATE = 42


# Data import
We retrieve all the required data for the analysis.

In [38]:
df = pd.read_csv(RAW_FOLDER + 'train.csv', index_col=0)
df_evaluation = df.copy() 
df_evaluation.shape

(1460, 80)

# Data processing

## 1st experiment: the baseline
Here we will:

1. Drop the attributes with more than 80% nulls; and
1. Drop all non_numerical features.

### List of attributes to drop

In [66]:
lst_columns_null = serie_nulos(df_evaluation, corte=0.5).index.tolist()

lst_columns_null

4 atributos/features/campos possuem mais de 0.5 de valores nulos.


['PoolQC', 'MiscFeature', 'Alley', 'Fence']

### List of area attributes

In [67]:
lst_area = [x for x in df_evaluation.columns if ('area' in x.lower()) or ('sf' in x.lower())]

print(f"""There is/are {len(lst_area)} area attributes:
{lst_area}""")

There is/are 14 area attributes:
['LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'PoolArea']


### List of float attributes

In [68]:
lst_float = [
    x for x in df_evaluation.select_dtypes(include='float64').columns.tolist() if (x not in lst_area) and (x not in lst_time)
]

print(f"""There is/are {len(lst_float)} float attributes:
{lst_float}""")

There is/are 1 float attributes:
['LotFrontage']


### List of time attributes

With these features, I will create new time features that express the difference between the YrSold and the each other time feature. For example:

- 'YrSold' - 'YearBuilt' = 'YrSold_YearBuilt' -> express the building's age.

The difference will not be calculated between 'YrSold' and 'MoSold' because there's no meaning in doing this. 'MoSold' can be used later to observe if there's a seasonality within a year.

In [69]:
lst_time = [x for x in df_evaluation.columns if ('yr' in x.lower()) or ('year' in x.lower())]
# After reading the data description, I realized that 'MoSold' is a time attribute too.

print(f"""There is/are {len(lst_time)} time attributes:
{lst_time}""")

lst_time.remove('YrSold')

There is/are 4 time attributes:
['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']


In [79]:
def create_time_dif_columns(df: pd.DataFrame, lst: List) -> pd.DataFrame:
    df_temp = df.copy()

    for attribute in lst:
        df_temp[attribute + '_YrSold'] = df_temp['YrSold'] - df_temp[attribute]
        df_temp.drop(columns=attribute, inplace=True)
    
    return df_temp

df_test = create_time_dif_columns(df=df, lst=lst_time)

df_test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,YearBuilt_YrSold,YearRemodAdd_YrSold,GarageYrBlt_YrSold
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500,5,5,5.0
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500,31,31,31.0
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500,7,6,7.0
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000,91,36,8.0
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000,8,8,8.0


### Data output

In [89]:
df_output = df.copy()
df_output.drop(columns=lst_columns_null, inplace=True)

lst_final_columns = lst_area.copy()
lst_final_columns.extend(lst_time)
lst_final_columns.append('YrSold')

df_output = df_output[lst_final_columns]
df_output = create_time_dif_columns(df=df_output, lst=lst_time)

df_output.head()

,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,PoolArea,LotFrontage,YrSold,YearBuilt_YrSold,YearRemodAdd_YrSold,GarageYrBlt_YrSold
Id,,,,,,,,,,,,,,,,,,,
1,8450,196.0,706,0,150,856,856,854,0,1710,548,0,61,0,65.0,2008,5,5,5.0
2,9600,0.0,978,0,284,1262,1262,0,0,1262,460,298,0,0,80.0,2007,31,31,31.0
3,11250,162.0,486,0,434,920,920,866,0,1786,608,0,42,0,68.0,2008,7,6,7.0
4,9550,0.0,216,0,540,756,961,756,0,1717,642,0,35,0,60.0,2006,91,36,8.0
5,14260,350.0,655,0,490,1145,1145,1053,0,2198,836,192,84,0,84.0,2008,8,8,8.0


In [92]:
df_output.to_parquet(INTERIM_FOLDER + 'df_train_interim_01_baseline.pqt')